# Relaxed Search(제대로) 가미한 코드

### 5월 30일 업데이트

<strong> 이 코드 예상 실행 시간... 적으면 15분... 많으면 20분?(데스크탑 기준...)<br>데이터 양에 따라 차이가 남.</strong>



<strong> 참고로 밑에 스스로 판단해서 parameter 넣어하는 칸이 하나 있음.<br>한번 쭉 밀고 그 셀 채우고 다시 쭉 미셈</strong>
<br>
<strong> 아 그리고 XGboost 는 계속 Warning 이 뜨는데 이건 뭔 짓을 해도 뭐 없앨 수가 없네...</strong>

In [1]:
# 최적의 패러미터 가진 모델을 몇번 돌릴지 값.
test_times=20
# random 인 max_state 를 설정하기 위한 변수.
full_count=0 # 건들지 말것.
# test size
ts = 0.2
# alpha value (밑에 정규검사 등에서 이용되는 p 값)
alpha = 0.05

In [2]:
#데이터 파일 업로드
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from numpy import mean
from numpy import std
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy import stats
import xgboost as xgb
from scipy.stats import wilcoxon
import scipy.stats
import random
random.seed(42)
import warnings
np.set_printoptions(precision=4,suppress=True)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',500)
plt.rcParams["font.family"]='Malgun Gothic'

아래 셀에서 데이터 수정. <strong> 앞에 Data/ 있어야 제대로 됨</strong><br>
원하는 데이터 이름만 집어넣으면 뒤에는 알아서 될거임

In [3]:
dataname = "bundang3years_small1.csv"

In [4]:
#데이터 확인 및 필요 없는 열 삭제
pd.set_option('display.max_columns', 500)
df=pd.read_csv(dataname)
df.drop(['시군구'],axis=1,inplace=True)
df.drop(['단지명'],axis=1,inplace=True)
df.drop(['건축년도'],axis=1,inplace=True)

Y = df['거래금액'].values
Y = Y.astype('float')
X = df.drop(labels=['거래금액'],axis=1)
X = X.astype('float')

### 1. Random Forest

n_estimators = number of trees in the foreset<br>
max_features = max number of features considered for splitting a node<br>
max_depth = max number of levels in each decision tree<br>
min_samples_split = min number of data points placed in a node before the node is split<br>
min_samples_leaf = min number of data points allowed in a leaf node<br>
bootstrap = method for sampling data points (with or without replacement)

In [5]:
Tree_nums = [5,10,50,100,250,500,750]
Var_nums = [1,3,5,7]

# Random Forest Relaxed Search
x = len(Tree_nums)
y = len(Var_nums)
count=0

RF_RMSE_train_list =[]
RF_RMSE_test_list =[]
RF_MAPE_train_list =[]
RF_MAPE_test_list =[]
RF_par_diff_list=[]
RF_par_trees_list=[]
RF_par_var_list=[]
RF_min_vars_trees_list=[]
RF_min_vars_var_list=[]   

print("----progress...----")

for a in Tree_nums:
    for b in Var_nums:
        RF_temp_RMSE_train=0
        RF_temp_RMSE_test=0
        RF_temp_MAPE_train=0
        RF_temp_MAPE_test=0
        RF_temp_RMSE_diff=0
        for i in range(test_times):
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, random_state=full_count)
        
            RF_model = RandomForestRegressor(n_estimators=a, max_depth=b,random_state=full_count)
            RF_model.fit(X_train, Y_train)
        
            RF_train_test = RF_model.predict(X_train)
            RF_train_test = RF_train_test.astype('int')
            
            RF_prediction_test = RF_model.predict(X_test)
            RF_prediction_test = RF_prediction_test.astype('int')   
            
            RF_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_train_test)).astype('int')
            RF_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_prediction_test)).astype('int')
            RF_MAPE_train = round(np.mean(np.abs((Y_train - RF_train_test) / Y_train)) * 100 ,2)
            RF_MAPE_test = round(np.mean(np.abs((Y_test - RF_prediction_test) / Y_test)) * 100 ,2)
            RF_RMSE_diff = np.abs((RF_RMSE_test-RF_RMSE_train)/RF_RMSE_test *100)
            
            RF_temp_RMSE_train+=RF_RMSE_train
            RF_temp_RMSE_test+=RF_RMSE_test
            RF_temp_MAPE_train+=RF_MAPE_train
            RF_temp_MAPE_test+=RF_MAPE_test
            RF_temp_RMSE_diff+=RF_RMSE_diff
            full_count+=1
            count+=1
            progress = round(count/(x*y*test_times)*100,2)
            print(round(progress), end = ", ")
#             print(" {:.2f}% , case:{} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#             "---  train RMSE : {} , test RMSE : {}\n"
#             "---  train MAPE : {} , test MAPE : {}"
#                   .format(progress,count,ts,a,b,RF_RMSE_diff,RF_RMSE_train,RF_RMSE_test,RF_MAPE_train,RF_MAPE_test))
             
        RF_RMSE_train_list.append(RF_temp_RMSE_train/test_times)
        RF_RMSE_test_list.append(RF_temp_RMSE_test/test_times)
        RF_MAPE_train_list.append(RF_temp_MAPE_train/test_times)
        RF_MAPE_test_list.append(RF_temp_MAPE_test/test_times)
        RF_par_diff_list.append(RF_temp_RMSE_diff/test_times)
        RF_par_trees_list.append(a)
        RF_par_var_list.append(b)

for i in range(5):
    val = min(RF_par_diff_list)
    index = RF_par_diff_list.index(val)
    RF_par_diff_list.pop(index)
    RF_min_vars_trees_list.append(RF_par_trees_list.pop(index))
    RF_min_vars_var_list.append(RF_par_var_list.pop(index))
    
    
# RF_min_vars_trees_list
# RF_min_vars_var_list

----progress...----
0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40, 41, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 42, 43, 43, 43, 43, 43, 44, 44, 44, 44, 44, 44, 45, 45, 45, 45, 45, 46, 46, 46, 46, 4

In [6]:
print("...testing...")

progress=0
count=0
RF_Best_RMSE_train_list=[]
RF_Best_RMSE_test_list=[]
RF_Best_MAPE_train_list=[]
RF_Best_MAPE_test_list=[]
RF_Best_RMSE_diff_list=[]

RF_Best_total_data=[]

for i in range(5):
    temp_RF_Best_RMSE_train_list=[]
    temp_RF_Best_RMSE_test_list=[]
    temp_RF_Best_MAPE_train_list=[]
    temp_RF_Best_MAPE_test_list=[]
    temp_RF_Best_RMSE_diff_list=[]
    for k in range(test_times):
        a = RF_min_vars_trees_list[i]
        b = RF_min_vars_var_list[i]

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts,random_state=full_count)

        RF_Best_model = RandomForestRegressor(n_estimators=a, max_depth=b,random_state=full_count )
        RF_Best_model.fit(X_train, Y_train)

        RF_Best_prediction_test = RF_Best_model.predict(X_test)
        RF_Best_prediction_test = RF_Best_prediction_test.astype('int')

        RF_Best_train_test = RF_Best_model.predict(X_train)
        RF_Best_train_test = RF_Best_train_test.astype('int')

        RF_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_Best_train_test)).astype('int')
        RF_Best_MAPE_train = round(np.mean(np.abs((Y_train - RF_Best_train_test) / Y_train)) * 100 ,2)

        RF_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_Best_prediction_test)).astype('int')
        RF_Best_MAPE_test = round(np.mean(np.abs((Y_test - RF_Best_prediction_test) / Y_test)) * 100 ,2)

        RF_Best_RMSE_diff = np.abs((RF_Best_RMSE_test-RF_Best_RMSE_train)/RF_Best_RMSE_test *100)
        
        temp_RF_Best_RMSE_train_list.append(RF_Best_RMSE_train)
        temp_RF_Best_RMSE_test_list.append(RF_Best_RMSE_test)
        temp_RF_Best_MAPE_train_list.append(RF_Best_MAPE_train)
        temp_RF_Best_MAPE_test_list.append(RF_Best_MAPE_test)
        temp_RF_Best_RMSE_diff_list.append(RF_Best_RMSE_diff)
        
        full_count+=1
        count+=1
        progress = round(count/(10*5)*100,2)
#         print(" {:.2f}% , case:{} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#                 "---  train RMSE : {} , test RMSE : {}\n"
#                 "---  train MAPE : {} , test MAPE : {}"
#                       .format(progress,count,ts,a,b,RF_Best_RMSE_diff,RF_Best_RMSE_train,RF_Best_RMSE_test,RF_Best_MAPE_train,RF_Best_MAPE_test))
    
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_diff_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_diff_list))


...testing...


In [7]:
print("PARAMETERS FOR BEST RF...")
for i in range(5):
    print("Tree_num : {}, Var_num : {}".format(RF_min_vars_trees_list[i],RF_min_vars_var_list[i]))

rf1=[]
rf2=[]
rf3=[]
rf4=[]
rf5=[]

for i in range(50):
    k=i//10
    if k==0:
        rf1.append(RF_Best_total_data[i])
    if k==1:
        rf2.append(RF_Best_total_data[i])
    if k==2:
        rf3.append(RF_Best_total_data[i])
    if k==3:
        rf4.append(RF_Best_total_data[i])
    if k==4:
        rf5.append(RF_Best_total_data[i])
    
RF_s1 = pd.Series(rf1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

RF_s2 = pd.Series(rf2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
RF_s3 = pd.Series(rf3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
RF_s4 = pd.Series(rf4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
RF_s5 = pd.Series(rf5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

RF_best_total_dataframe = pd.concat([RF_s1,RF_s2,RF_s3,RF_s4,RF_s5],axis=1)
RF_best_total_dataframe

PARAMETERS FOR BEST RF...
Tree_num : 10, Var_num : 3
Tree_num : 250, Var_num : 1
Tree_num : 5, Var_num : 1
Tree_num : 50, Var_num : 1
Tree_num : 750, Var_num : 3


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,3251.000000,6078.250000,6094.900000,6073.250000,3213.600000
RMSE test Mean,3373.500000,6058.300000,6045.550000,6052.550000,3271.100000
RMSE train Stdev,83.694086,36.697241,44.743603,41.578690,56.107397
RMSE test Stdev,161.819189,183.119114,184.313720,204.866170,142.637968
MAPE train Mean,11.426000,21.166500,21.181500,21.149500,11.340500
MAPE test Mean,11.654500,21.044500,21.152500,21.061000,11.461000
MAPE train Stdev,0.255253,0.150276,0.179340,0.129672,0.180955
MAPE test Stdev,0.523130,0.774141,0.682282,0.603075,0.430708
RMSE diff(%) Mean,4.432673,2.857289,2.916275,2.976400,3.704085
RMSE diff(%) Stdev,3.657913,2.252782,2.408484,2.951530,2.834167


### 2. Support Vector Machine parameters

In [8]:
gamma_nums = [0.00005,0.0001,0.000125,0.00015,0.000175,0.0002,]
C_values= [5000,75000,10000,12500,15000]
x = len(gamma_nums)
y = len(C_values)
count=0

SVR_RMSE_train_list =[]
SVR_RMSE_test_list =[]
SVR_MAPE_train_list =[]
SVR_MAPE_test_list =[]
SVR_par_diff_list=[]
SVR_par_gamma_list=[]
SVR_par_cval_list=[]
SVR_min_vars_gamma_list=[]
SVR_min_vars_cval_list=[]   

print("----progress...----")

for a in gamma_nums:
    for b in C_values:
        SVR_temp_RMSE_train=0
        SVR_temp_RMSE_test=0
        SVR_temp_MAPE_train=0
        SVR_temp_MAPE_test=0
        SVR_temp_RMSE_diff=0
        for i in range(test_times): 
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts,random_state=full_count)

            SVR_model = svm.SVR(kernel='rbf',gamma=a,C=b)
            SVR_model.fit(X_train, Y_train)
            
            SVR_prediction_test = SVR_model.predict(X_test)
            SVR_prediction_test = SVR_prediction_test.astype('int')
            
            SVR_train_test = SVR_model.predict(X_train)
            SVR_train_test = SVR_train_test.astype('int')
            
            SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
            SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
            SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
            SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
            SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)

            SVR_temp_RMSE_train+=SVR_RMSE_train
            SVR_temp_RMSE_test+=SVR_RMSE_test
            SVR_temp_MAPE_train+=SVR_MAPE_train
            SVR_temp_MAPE_test+=SVR_MAPE_test
            SVR_temp_RMSE_diff+=SVR_RMSE_diff
            
            full_count+=1
            count+=1
            progress = round(count/(x*y*test_times)*100,2)
            print(round(progress), end = ", ")
#             print(" {:.2f}% , case:{} , Gamma_size: {} ,  C_values: {} , difference(%) : {:.2f}\n "
#             "---  train RMSE : {} , test RMSE : {}\n"
#             "---  train MAPE : {} , test MAPE : {}"
#                   .format(progress,count,a,b,SVR_RMSE_diff,SVR_RMSE_train,SVR_RMSE_test,SVR_MAPE_train,SVR_MAPE_test))            
            
        SVR_RMSE_train_list.append(SVR_RMSE_train)
        SVR_RMSE_test_list.append(SVR_RMSE_test)
        SVR_MAPE_train_list.append(SVR_MAPE_train)
        SVR_MAPE_test_list.append(SVR_MAPE_test)
        SVR_par_diff_list.append(SVR_RMSE_diff)
        SVR_par_gamma_list.append(a)
        SVR_par_cval_list.append(b) 

for i in range(5):
    val = min(SVR_par_diff_list)
    index = SVR_par_diff_list.index(val)
    SVR_par_diff_list.pop(index)
    SVR_min_vars_gamma_list.append(SVR_par_gamma_list.pop(index))
    SVR_min_vars_cval_list.append(SVR_par_cval_list.pop(index))

----progress...----
0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 34, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40, 40, 40, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 42, 42, 43, 43, 43, 43, 

In [9]:
print("\n\n...testing...\n")
progress=0
count=0
SVR_Best_RMSE_train_list=[]
SVR_Best_RMSE_test_list=[]
SVR_Best_MAPE_train_list=[]
SVR_Best_MAPE_test_list=[]
SVR_Best_RMSE_diff_list=[]

SVR_Best_total_data=[]

for i in range(5):
    temp_SVR_Best_RMSE_train_list=[]
    temp_SVR_Best_RMSE_test_list=[]
    temp_SVR_Best_MAPE_train_list=[]
    temp_SVR_Best_MAPE_test_list=[]
    temp_SVR_Best_RMSE_diff_list=[]
    for k in range(test_times):
        a = SVR_min_vars_gamma_list[i]
        b = SVR_min_vars_cval_list[i]

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts,random_state=full_count)

        SVR_Best_model = svm.SVR(kernel='rbf',gamma=a,C=b)
        SVR_Best_model.fit(X_train, Y_train)
            
        SVR_Best_prediction_test = SVR_Best_model.predict(X_test)
        SVR_Best_prediction_test = SVR_Best_prediction_test.astype('int')
            
        SVR_Best_train_test = SVR_Best_model.predict(X_train)
        SVR_Best_train_test = SVR_Best_train_test.astype('int')
            
        SVR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_Best_train_test)).astype('int')
        SVR_Best_MAPE_train = round(np.mean(np.abs((Y_train - SVR_Best_train_test) / Y_train)) * 100 ,2)
            
        SVR_Best_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_Best_prediction_test)).astype('int')
        SVR_Best_MAPE_test = round(np.mean(np.abs((Y_test - SVR_Best_prediction_test) / Y_test)) * 100 ,2)
            
        SVR_Best_RMSE_diff = np.abs((SVR_Best_RMSE_test-SVR_Best_RMSE_train)/SVR_Best_RMSE_test *100)
        
        temp_SVR_Best_RMSE_train_list.append(SVR_Best_RMSE_train)
        temp_SVR_Best_RMSE_test_list.append(SVR_Best_RMSE_test)
        temp_SVR_Best_MAPE_train_list.append(SVR_Best_MAPE_train)
        temp_SVR_Best_MAPE_test_list.append(SVR_Best_MAPE_test)
        temp_SVR_Best_RMSE_diff_list.append(SVR_Best_RMSE_diff)
        full_count+=1  
        count+=1
#         progress = round(count/(test_times*5)*100,2)
#         print(" {:.2f}% , case:{} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#                 "---  train RMSE : {} , test RMSE : {}\n"
#                 "---  train MAPE : {} , test MAPE : {}"
#                       .format(progress,count,ts,a,b,SVR_Best_RMSE_diff,SVR_Best_RMSE_train,SVR_Best_RMSE_test,SVR_Best_MAPE_train,SVR_Best_MAPE_test))
    
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_diff_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_diff_list))

    
    




...testing...



In [10]:
print("PARAMETERS FOR BEST SVR...")
for i in range(5):
    print("Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))

SVR1=[]
SVR2=[]
SVR3=[]
SVR4=[]
SVR5=[]

for i in range(50):
    k=i//10
    if k==0:
        SVR1.append(SVR_Best_total_data[i])
    if k==1:
        SVR2.append(SVR_Best_total_data[i])
    if k==2:
        SVR3.append(SVR_Best_total_data[i])
    if k==3:
        SVR4.append(SVR_Best_total_data[i])
    if k==4:
        SVR5.append(SVR_Best_total_data[i])

    
SVR_s1 = pd.Series(SVR1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

SVR_s2 = pd.Series(SVR2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
SVR_s3 = pd.Series(SVR3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
SVR_s4 = pd.Series(SVR4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
SVR_s5 = pd.Series(SVR5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

SVR_best_total_dataframe_rbf = pd.concat([SVR_s1,SVR_s2,SVR_s3,SVR_s4,SVR_s5],axis=1)
SVR_best_total_dataframe_rbf

PARAMETERS FOR BEST SVR...
Gamma_size: 0.0002 ,  C_values: 12500
Gamma_size: 5e-05 ,  C_values: 15000
Gamma_size: 0.00015 ,  C_values: 12500
Gamma_size: 0.000125 ,  C_values: 15000
Gamma_size: 0.0001 ,  C_values: 15000


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,1813.150000,2559.250000,1948.750000,1948.850000,2067.800000
RMSE test Mean,1950.200000,2671.150000,2105.550000,1976.350000,2252.150000
RMSE train Stdev,32.998901,50.470660,32.684668,34.272839,32.743854
RMSE test Stdev,133.250366,267.607600,209.428144,194.603000,239.639370
MAPE train Mean,4.291500,5.591500,4.468000,4.515000,4.671500
MAPE test Mean,4.876500,5.933000,5.118500,4.896500,5.233500
MAPE train Stdev,0.055612,0.067028,0.035299,0.038794,0.059184
MAPE test Stdev,0.298283,0.356428,0.259524,0.220687,0.273172
RMSE diff(%) Mean,7.947237,8.203167,7.418439,7.459402,10.567605
RMSE diff(%) Stdev,4.892510,6.917234,7.679612,6.024688,6.740921


### 3. XGBoost ... <br>
DAMN...

In [11]:
feattree=[0.2,0.4,0.6]
learnrate=[0.1,0.2,0.3,0.4]
maxdepth=[5,7,9,11]
trees=[50,75,100,250,500]
count=0

XGB_RMSE_train_list =[]
XGB_RMSE_test_list =[]
XGB_MAPE_train_list =[]
XGB_MAPE_test_list =[]
XGB_par_diff_list=[]

XGB_par_feattree_list=[]
XGB_par_learnrate_list=[]
XGB_par_maxdepth_list=[]
XGB_par_trees_list=[]

XGB_min_vars_feattree_list=[]
XGB_min_vars_learnrate_list=[]  
XGB_min_vars_maxdepth_list=[]
XGB_min_vars_trees_list=[] 

print("----progress...----")

for a in feattree:
    for b in learnrate:
        for c in maxdepth:
            for d in trees:
                X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, random_state=full_count)
                xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = a, 
                                              learning_rate = b, max_depth = c, n_estimators = d)
                xg_reg.fit(X_train,Y_train)
                    
                XGB_train_test = xg_reg.predict(X_train).astype('int')
                XGB_prediction_test = xg_reg.predict(X_test).astype('int')
                    
                XGB_RMSE_train =np.sqrt(mean_squared_error(Y_train, XGB_train_test)).astype('int')
                XGB_RMSE_test = np.sqrt(mean_squared_error(Y_test, XGB_prediction_test)).astype('int')
                XGB_MAPE_train = round(np.mean(np.abs((Y_train - XGB_train_test) / Y_train)) * 100 ,2)
                XGB_MAPE_test = round(np.mean(np.abs((Y_test - XGB_prediction_test) / Y_test)) * 100 ,2)
                XGB_RMSE_diff = np.abs((XGB_RMSE_test-XGB_RMSE_train)/XGB_RMSE_test *100)

                full_count+=1
                count+=1
                    
#                 progress = round(count/(len(feattree)*len(learnrate)*len(maxdepth)*len(trees))*100,2)
#                 print(round(progress), end = ", ")
                
#                 print("{} {} {} {} \n{} {} {} {} {}".
#                     format(a, b, c,d,
#                         XGB_RMSE_train,XGB_RMSE_test,XGB_MAPE_train,XGB_MAPE_test,round(XGB_RMSE_diff,2) ))
                
                
                XGB_RMSE_train_list.append(XGB_RMSE_train)
                XGB_RMSE_test_list.append(XGB_RMSE_test)
                XGB_MAPE_train_list.append(XGB_MAPE_train)
                XGB_MAPE_test_list.append(XGB_MAPE_test)
                XGB_par_diff_list.append(XGB_RMSE_diff)
                
                XGB_par_feattree_list.append(a)
                XGB_par_learnrate_list.append(b)
                XGB_par_maxdepth_list.append(c)
                XGB_par_trees_list.append(d)

sp_count=0
while sp_count < 5:
    val = min(XGB_par_diff_list)
    index = XGB_par_diff_list.index(val)
    XGB_par_diff_list.pop(index)
    if((XGB_MAPE_train_list[index])<=15 and (XGB_MAPE_test_list[index])<=15):
        XGB_min_vars_feattree_list.append(XGB_par_feattree_list.pop(index))
        XGB_min_vars_learnrate_list.append(XGB_par_learnrate_list.pop(index))
        XGB_min_vars_maxdepth_list.append(XGB_par_maxdepth_list.pop(index))
        XGB_min_vars_trees_list.append(XGB_par_trees_list.pop(index))
        sp_count+=1

----progress...----
[15:47:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:47:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:47:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:47:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:47:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:47:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/

In [12]:
print("PARAMETERS FOR BEST XGB...")
for i in range(5):
    print(" feature % : {}, learn_rate : {}, max_depth : {}, trees : {} ".
          format(XGB_min_vars_feattree_list[i],XGB_min_vars_learnrate_list[i],XGB_min_vars_maxdepth_list[i],XGB_min_vars_trees_list[i]))

print("...testing...")

progress=0
count=0
XGB_Best_RMSE_train_list=[]
XGB_Best_RMSE_test_list=[]
XGB_Best_MAPE_train_list=[]
XGB_Best_MAPE_test_list=[]
XGB_Best_RMSE_diff_list=[]

XGB_Best_total_data=[]

for i in range(5):
    temp_XGB_Best_RMSE_train_list=[]
    temp_XGB_Best_RMSE_test_list=[]
    temp_XGB_Best_MAPE_train_list=[]
    temp_XGB_Best_MAPE_test_list=[]
    temp_XGB_Best_RMSE_diff_list=[]
    for k in range(test_times):
        a = XGB_min_vars_feattree_list[i]
        b = XGB_min_vars_learnrate_list[i]
        c = XGB_min_vars_maxdepth_list[i]
        d = XGB_min_vars_trees_list[i]

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, random_state=full_count)
        xg_Best_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = a, 
                                learning_rate = b, max_depth = c, n_estimators = d)
        
        xg_Best_reg.fit(X_train,Y_train)
                    
        XGB_Best_train_test = xg_Best_reg.predict(X_train).astype('int')
        XGB_Best_prediction_test = xg_Best_reg.predict(X_test).astype('int')
                    
        XGB_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, XGB_Best_train_test)).astype('int')
        XGB_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, XGB_Best_prediction_test)).astype('int')

        XGB_Best_RMSE_train = np.sqrt(mean_squared_error(Y_train, XGB_Best_train_test)).astype('int')
        XGB_Best_MAPE_train = round(np.mean(np.abs((Y_train - XGB_Best_train_test) / Y_train)) * 100 ,2)

        XGB_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, XGB_Best_prediction_test)).astype('int')
        XGB_Best_MAPE_test = round(np.mean(np.abs((Y_test - XGB_Best_prediction_test) / Y_test)) * 100 ,2)

        XGB_Best_RMSE_diff = np.abs((XGB_Best_RMSE_test-XGB_Best_RMSE_train)/XGB_Best_RMSE_test *100)
        
        temp_XGB_Best_RMSE_train_list.append(XGB_Best_RMSE_train)
        temp_XGB_Best_RMSE_test_list.append(XGB_Best_RMSE_test)
        temp_XGB_Best_MAPE_train_list.append(XGB_Best_MAPE_train)
        temp_XGB_Best_MAPE_test_list.append(XGB_Best_MAPE_test)
        temp_XGB_Best_RMSE_diff_list.append(XGB_Best_RMSE_diff)
        
        full_count+=1
        count+=1
        progress = round(count/(5*test_times)*100,2)

    XGB_Best_total_data.append(mean(temp_XGB_Best_RMSE_train_list))
    XGB_Best_total_data.append(mean(temp_XGB_Best_RMSE_test_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_RMSE_train_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_RMSE_test_list))
    XGB_Best_total_data.append(mean(temp_XGB_Best_MAPE_train_list))
    XGB_Best_total_data.append(mean(temp_XGB_Best_MAPE_test_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_MAPE_train_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_MAPE_test_list))
    XGB_Best_total_data.append(mean(temp_XGB_Best_RMSE_diff_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_RMSE_diff_list))


PARAMETERS FOR BEST XGB...
 feature % : 0.4, learn_rate : 0.1, max_depth : 5, trees : 100 
 feature % : 0.2, learn_rate : 0.1, max_depth : 5, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 7, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 5, trees : 75 
 feature % : 0.2, learn_rate : 0.2, max_depth : 5, trees : 75 
...testing...
[15:50:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:50:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:50:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:50:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu

In [13]:
print("PARAMETERS FOR BEST XGB...")
for i in range(5):
    print(" feature % : {}, learn_rate : {}, max_depth : {}, trees : {} ".
          format(XGB_min_vars_feattree_list[i],XGB_min_vars_learnrate_list[i],XGB_min_vars_maxdepth_list[i],XGB_min_vars_trees_list[i]))

XGB1=[]
XGB2=[]
XGB3=[]
XGB4=[]
XGB5=[]

for i in range(50):
    k=i//10
    if k==0:
        XGB1.append(XGB_Best_total_data[i])
    if k==1:
        XGB2.append(XGB_Best_total_data[i])
    if k==2:
        XGB3.append(XGB_Best_total_data[i])
    if k==3:
        XGB4.append(XGB_Best_total_data[i])
    if k==4:
        XGB5.append(XGB_Best_total_data[i])
    
XGB_s1 = pd.Series(XGB1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

XGB_s2 = pd.Series(XGB2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
XGB_s3 = pd.Series(XGB3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
XGB_s4 = pd.Series(XGB4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
XGB_s5 = pd.Series(XGB5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')


XGB_best_total_dataframe = pd.concat([XGB_s1,XGB_s2,XGB_s3,XGB_s4,XGB_s5],axis=1)
print("PARAMETERS FOR BEST XGB...")
XGB_best_total_dataframe

PARAMETERS FOR BEST XGB...
 feature % : 0.4, learn_rate : 0.1, max_depth : 5, trees : 100 
 feature % : 0.2, learn_rate : 0.1, max_depth : 5, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 7, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 5, trees : 75 
 feature % : 0.2, learn_rate : 0.2, max_depth : 5, trees : 75 
PARAMETERS FOR BEST XGB...


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,1125.250000,3085.150000,2887.300000,2157.550000,1502.900000
RMSE test Mean,1530.850000,3428.350000,3279.850000,2534.700000,1992.800000
RMSE train Stdev,27.881670,26.414532,23.469342,25.805959,28.167179
RMSE test Stdev,112.105430,189.267872,155.146793,163.349962,160.672835
MAPE train Mean,3.565000,8.406500,7.741000,6.279500,4.645500
MAPE test Mean,4.539000,9.166500,8.772000,7.220500,5.789500
MAPE train Stdev,0.054176,0.059184,0.076413,0.084704,0.059201
MAPE test Stdev,0.168550,0.268016,0.298523,0.280668,0.233805
RMSE diff(%) Mean,25.989801,9.717195,11.746848,14.474081,23.996597
RMSE diff(%) Stdev,6.949341,5.357057,4.721592,6.329931,7.409729


# COMPARING ALL FOUR IN ONE

In [14]:
print("PARAMETERS FOR BEST RF...")
for i in range(5):
    print("Tree_num : {}, Var_num : {}".format(RF_min_vars_trees_list[i],RF_min_vars_var_list[i]))
RF_best_total_dataframe

PARAMETERS FOR BEST RF...
Tree_num : 10, Var_num : 3
Tree_num : 250, Var_num : 1
Tree_num : 5, Var_num : 1
Tree_num : 50, Var_num : 1
Tree_num : 750, Var_num : 3


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,3251.000000,6078.250000,6094.900000,6073.250000,3213.600000
RMSE test Mean,3373.500000,6058.300000,6045.550000,6052.550000,3271.100000
RMSE train Stdev,83.694086,36.697241,44.743603,41.578690,56.107397
RMSE test Stdev,161.819189,183.119114,184.313720,204.866170,142.637968
MAPE train Mean,11.426000,21.166500,21.181500,21.149500,11.340500
MAPE test Mean,11.654500,21.044500,21.152500,21.061000,11.461000
MAPE train Stdev,0.255253,0.150276,0.179340,0.129672,0.180955
MAPE test Stdev,0.523130,0.774141,0.682282,0.603075,0.430708
RMSE diff(%) Mean,4.432673,2.857289,2.916275,2.976400,3.704085
RMSE diff(%) Stdev,3.657913,2.252782,2.408484,2.951530,2.834167


In [15]:
print("PARAMETERS FOR BEST SVR...")
for i in range(5):
    print("Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))
SVR_best_total_dataframe_rbf

PARAMETERS FOR BEST SVR...
Gamma_size: 0.0002 ,  C_values: 12500
Gamma_size: 5e-05 ,  C_values: 15000
Gamma_size: 0.00015 ,  C_values: 12500
Gamma_size: 0.000125 ,  C_values: 15000
Gamma_size: 0.0001 ,  C_values: 15000


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,1813.150000,2559.250000,1948.750000,1948.850000,2067.800000
RMSE test Mean,1950.200000,2671.150000,2105.550000,1976.350000,2252.150000
RMSE train Stdev,32.998901,50.470660,32.684668,34.272839,32.743854
RMSE test Stdev,133.250366,267.607600,209.428144,194.603000,239.639370
MAPE train Mean,4.291500,5.591500,4.468000,4.515000,4.671500
MAPE test Mean,4.876500,5.933000,5.118500,4.896500,5.233500
MAPE train Stdev,0.055612,0.067028,0.035299,0.038794,0.059184
MAPE test Stdev,0.298283,0.356428,0.259524,0.220687,0.273172
RMSE diff(%) Mean,7.947237,8.203167,7.418439,7.459402,10.567605
RMSE diff(%) Stdev,4.892510,6.917234,7.679612,6.024688,6.740921


In [16]:
for i in range(5):
    print(" feature % : {}, learn_rate : {}, max_depth : {}, trees : {} ".
          format(XGB_min_vars_feattree_list[i],XGB_min_vars_learnrate_list[i],XGB_min_vars_maxdepth_list[i],XGB_min_vars_trees_list[i]))

XGB_best_total_dataframe

 feature % : 0.4, learn_rate : 0.1, max_depth : 5, trees : 100 
 feature % : 0.2, learn_rate : 0.1, max_depth : 5, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 7, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 5, trees : 75 
 feature % : 0.2, learn_rate : 0.2, max_depth : 5, trees : 75 


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,1125.250000,3085.150000,2887.300000,2157.550000,1502.900000
RMSE test Mean,1530.850000,3428.350000,3279.850000,2534.700000,1992.800000
RMSE train Stdev,27.881670,26.414532,23.469342,25.805959,28.167179
RMSE test Stdev,112.105430,189.267872,155.146793,163.349962,160.672835
MAPE train Mean,3.565000,8.406500,7.741000,6.279500,4.645500
MAPE test Mean,4.539000,9.166500,8.772000,7.220500,5.789500
MAPE train Stdev,0.054176,0.059184,0.076413,0.084704,0.059201
MAPE test Stdev,0.168550,0.268016,0.298523,0.280668,0.233805
RMSE diff(%) Mean,25.989801,9.717195,11.746848,14.474081,23.996597
RMSE diff(%) Stdev,6.949341,5.357057,4.721592,6.329931,7.409729


아래 셀은 값들을 직접 입력할것.<br>
위에 표 보고 제일 괜찮을 것 같은거 스스로 판단해서<br>
그 parameter 들을 아래에다 넣자.<br>
그리고 다시 쭉 밀면 된다.<br>

In [5]:
RF_trees= 750
RF_depth= 3
SVR_gamma= 0.000125
SVR_c= 15000
XGB_colsample=0.4
XGB_learn_rate=0.1
XGB_max_depth=5
XGB_trees=100

In [6]:
final_RF_RMSE_train=[]
final_RF_RMSE_test=[]
final_RF_MAPE_train=[]
final_RF_MAPE_test=[]
final_RF_RMSE_diff=[]

final_SVR_RMSE_train=[]
final_SVR_RMSE_test=[]
final_SVR_MAPE_train=[]
final_SVR_MAPE_test=[]
final_SVR_RMSE_diff=[]

final_XGB_RMSE_train=[]
final_XGB_RMSE_test=[]
final_XGB_MAPE_train=[]
final_XGB_MAPE_test=[]
final_XGB_RMSE_diff=[]

final_LR_RMSE_train=[]
final_LR_RMSE_test=[]
final_LR_MAPE_train=[]
final_LR_MAPE_test=[]
final_LR_RMSE_diff=[]

count=0
for i in range(test_times):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, random_state=full_count)
    
    # RF
    
    RF_model = RandomForestRegressor(n_estimators=RF_trees, max_depth=RF_depth,random_state=full_count)
    RF_model.fit(X_train, Y_train)      
    RF_prediction_test = RF_model.predict(X_test).astype('int')  
    RF_train_test = RF_model.predict(X_train).astype('int') 
    
    RF_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_train_test)).astype('int')
    RF_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_prediction_test)).astype('int')
    RF_MAPE_train = round(np.mean(np.abs((Y_train - RF_train_test) / Y_train)) * 100 ,2) 
    RF_MAPE_test = round(np.mean(np.abs((Y_test - RF_prediction_test) / Y_test)) * 100 ,2)        
    RF_RMSE_diff = np.abs((RF_RMSE_test-RF_RMSE_train)/RF_RMSE_test *100)      
    
    final_RF_RMSE_train.append(RF_RMSE_train)
    final_RF_RMSE_test.append(RF_RMSE_test)
    final_RF_MAPE_train.append(RF_MAPE_train)
    final_RF_MAPE_test.append(RF_MAPE_test)
    final_RF_RMSE_diff.append(RF_RMSE_diff)
    
    full_count+=1
    
    # SVR
    
    SVR_model = svm.SVR(kernel='rbf',gamma=SVR_gamma,C=SVR_c)
    SVR_model.fit(X_train, Y_train)
    SVR_prediction_test = SVR_model.predict(X_test).astype('int')       
    SVR_train_test = SVR_model.predict(X_train).astype('int')
    
    SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
    SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
    SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
    SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
    SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)
            
    final_SVR_RMSE_train.append(SVR_RMSE_train)
    final_SVR_RMSE_test.append(SVR_RMSE_test)
    final_SVR_MAPE_train.append(SVR_MAPE_train)
    final_SVR_MAPE_test.append(SVR_MAPE_test)        
    final_SVR_RMSE_diff.append(SVR_RMSE_diff)
    
    full_count+=1
        
    # XGboost 
        
    xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = XGB_colsample, learning_rate = XGB_learn_rate, 
                              max_depth = XGB_max_depth, n_estimators = XGB_trees)
    xg_reg.fit(X_train,Y_train)
    
    XGB_train_test = xg_reg.predict(X_train).astype('int')
    XGB_prediction_test = xg_reg.predict(X_test).astype('int')
                    
    XGB_RMSE_train =np.sqrt(mean_squared_error(Y_train, XGB_train_test)).astype('int')
    XGB_RMSE_test = np.sqrt(mean_squared_error(Y_test, XGB_prediction_test)).astype('int')
    XGB_MAPE_train = round(np.mean(np.abs((Y_train - XGB_train_test) / Y_train)) * 100 ,2)
    XGB_MAPE_test = round(np.mean(np.abs((Y_test - XGB_prediction_test) / Y_test)) * 100 ,2)
    XGB_RMSE_diff = np.abs((XGB_RMSE_test-XGB_RMSE_train)/XGB_RMSE_test *100)
    
    final_XGB_RMSE_train.append(XGB_RMSE_train)
    final_XGB_RMSE_test.append(XGB_RMSE_test)
    final_XGB_MAPE_train.append(XGB_MAPE_train)
    final_XGB_MAPE_test.append(XGB_MAPE_test)
    final_XGB_RMSE_diff.append(XGB_RMSE_diff)    

    full_count+=1

    # LR
    
    LR_model=LinearRegression()
    LR_model.fit(X_train, Y_train)
    LR_prediction_test = LR_model.predict(X_test).astype('int')
    LR_train_test = LR_model.predict(X_train).astype('int')

    
    LR_RMSE_train =np.sqrt(mean_squared_error(Y_train, LR_train_test)).astype('int')
    LR_RMSE_test =np.sqrt(mean_squared_error(Y_test, LR_prediction_test)).astype('int')    
    LR_MAPE_train = round(np.mean(np.abs((Y_train - LR_train_test) / Y_train)) * 100 ,2)
    LR_MAPE_test = round(np.mean(np.abs((Y_test - LR_prediction_test) / Y_test)) * 100 ,2)            
    LR_RMSE_diff = np.abs((LR_RMSE_test-LR_RMSE_train)/LR_RMSE_test *100)
    
    final_LR_RMSE_train.append(LR_RMSE_train)
    final_LR_RMSE_test.append(LR_RMSE_test)
    final_LR_MAPE_train.append(LR_MAPE_train)
    final_LR_MAPE_test.append(LR_MAPE_test)    
    final_LR_RMSE_diff.append(LR_RMSE_diff)
    
    full_count+=1


[20:34:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:34:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:34:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:34:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:34:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:34:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:17

In [7]:
print("RMSE values")
RMSE_test_all = pd.concat([pd.Series(final_RF_RMSE_test),pd.Series(final_SVR_RMSE_test),
                           pd.Series(final_XGB_RMSE_test),pd.Series(final_LR_RMSE_test),pd.Series(final_LR_RMSE_train)],axis=1)

RMSE_test_all = RMSE_test_all.rename({'0':'RF'},axis='columns')
RMSE_test_all.columns=['RF','SVR','XGB','LR','train']
RMSE_test_all

RMSE values


,RF,SVR,XGB,LR,train
0,3108,2368,1493,3535,3403
1,3329,2156,1591,3638,3374
2,3465,2303,1335,3422,3430
3,3375,1815,1542,3488,3414
4,2942,1983,1671,3401,3435
5,3290,2450,1476,3557,3395
6,3320,2269,1538,3696,3358
7,3170,2297,1705,3504,3410
8,3572,1976,1469,3566,3393
9,3279,2064,1435,3635,3378


In [20]:
RF_RMSE_mean = mean(RMSE_test_all['RF'])
SVR_RMSE_mean = mean(RMSE_test_all['SVR'])
XGB_RMSE_mean = mean(RMSE_test_all['XGB'])
LR_RMSE_mean = mean(RMSE_test_all['LR'])
print("RF_RMSE_mean : {} \nSVR_RMSE_mean : {} \nXGB_RMSE_mean : {} \nLR_RMSE_mean : {}".
      format(RF_RMSE_mean,SVR_RMSE_mean,XGB_RMSE_mean,LR_RMSE_mean))

print("max: {}".format(max(RF_RMSE_mean,SVR_RMSE_mean,XGB_RMSE_mean,LR_RMSE_mean)))


RF_RMSE_mean : 3285.25 
SVR_RMSE_mean : 2099.8 
XGB_RMSE_mean : 1549.2 
LR_RMSE_mean : 3484.9
max: 3484.9


In [8]:
print("MAPE values")
MAPE_test_all = pd.concat([pd.Series(final_RF_MAPE_test),pd.Series(final_SVR_MAPE_test),
                           pd.Series(final_XGB_MAPE_test),pd.Series(final_LR_MAPE_test),pd.Series(final_LR_MAPE_train)],axis=1)

MAPE_test_all = MAPE_test_all.rename({'0':'RF'},axis='columns')
MAPE_test_all.columns=['RF','SVR','XGB','LR','train']
MAPE_test_all

MAPE values


,RF,SVR,XGB,LR,train
0,11.11,5.00,4.53,12.19,11.89
1,11.44,5.21,4.64,12.28,11.93
2,11.49,5.06,4.09,11.76,11.96
3,11.72,4.93,4.65,12.12,11.91
4,10.27,5.00,4.56,12.15,11.94
5,11.57,5.24,4.61,13.01,11.85
6,11.82,5.42,4.79,12.69,11.81
7,11.09,4.95,4.59,11.83,11.97
8,12.38,5.18,4.49,12.28,11.96
9,11.35,5.24,4.33,11.85,11.86


In [22]:
RF_MAPE_mean = mean(MAPE_test_all['RF'])
SVR_MAPE_mean = mean(MAPE_test_all['SVR'])
XGB_MAPE_mean = mean(MAPE_test_all['XGB'])
LR_MAPE_mean = mean(MAPE_test_all['LR'])
print("RF_MAPE_mean : {} \nSVR_MAPE_mean : {} \nXGB_MAPE_mean : {} \nLR_MAPE_mean : {}".
      format(RF_MAPE_mean,SVR_MAPE_mean,XGB_MAPE_mean,LR_MAPE_mean))

print("max: {}".format(max(RF_MAPE_mean,SVR_MAPE_mean,XGB_MAPE_mean,LR_MAPE_mean)))

RF_MAPE_mean : 11.405 
SVR_MAPE_mean : 5.03 
XGB_MAPE_mean : 4.4835 
LR_MAPE_mean : 12.211
max: 12.211


In [23]:
print("diff values")
diff_test_all = pd.concat([pd.Series(final_RF_RMSE_diff),pd.Series(final_SVR_RMSE_diff),
                           pd.Series(final_XGB_RMSE_diff),pd.Series(final_LR_RMSE_diff)],axis=1)

diff_test_all = diff_test_all.rename({'0':'RF'},axis='columns')
diff_test_all.columns=['RF','SVR','XGB','LR']
diff_test_all


diff values


,RF,SVR,XGB,LR
0,1.314583,6.582155,23.465947,3.764506
1,6.544811,5.695889,33.393720,4.146685
2,4.248862,5.425532,30.205096,5.653021
3,0.570342,1.057082,27.615335,1.412918
4,1.327160,0.099950,13.452915,2.905983
5,0.538827,2.776323,20.279720,5.356056
6,5.129043,3.602151,19.176136,1.775673
7,9.037820,17.220801,25.959662,9.095831
8,1.752709,1.582278,20.226308,5.333706
9,2.995539,15.507075,17.523534,3.078302


In [24]:
RF_diff_mean = mean(diff_test_all['RF'])
SVR_diff_mean = mean(diff_test_all['SVR'])
XGB_diff_mean = mean(diff_test_all['XGB'])
LR_diff_mean = mean(diff_test_all['LR'])
print("RF_diff_mean : {} \nSVR_diff_mean : {} \nXGB_diff_mean : {} \nLR_diff_mean : {}".
      format(RF_diff_mean,SVR_diff_mean,XGB_diff_mean,LR_diff_mean))

print("max: {}".format(max(RF_diff_mean,SVR_diff_mean,XGB_diff_mean,LR_diff_mean)))


RF_diff_mean : 3.682454880874188 
SVR_diff_mean : 11.035936510610506 
XGB_diff_mean : 27.21112995915424 
LR_diff_mean : 3.959955886373353
max: 27.21112995915424


In [25]:
# 정규 검정. 오직 하나의 줄, array 를 이용할 것
def Ntest(df):
    stat,p = stats.normaltest(df)
    if p>alpha:
        return False
    else:
        return True
    
# 그냥 4개를 정규검정해서 list 로 True False 반환
def fourNtest(k):
    a = k['RF']
    b = k['SVR']
    c = k['XGB']
    d = k['LR']
    tlist=[]
    tlist.append(Ntest(a))
    tlist.append(Ntest(b))
    tlist.append(Ntest(c))
    tlist.append(Ntest(d))
    return tlist

# T-test. Array 로 시킬 것.
def Ttest(df1,df2):
    s,p=stats.ttest_rel(df1, df2)
    if p>alpha:
        return False
    else:
        return True

# Wilcoxon test. Array 로 정큐분포를 이루지 않을때의 paired t-test
def Wiltest(df1,df2):
    z, p = wilcoxon(df2 - df1)
    if p>alpha:
        return False
    else:
        return True

In [26]:
print(fourNtest(RMSE_test_all))

[False, False, False, False]


In [27]:
print(fourNtest(MAPE_test_all))

[False, False, False, False]


In [28]:
print(fourNtest(diff_test_all))

[False, False, False, False]


그냥 다 정규분포 안 따른다고 생각해도 될듯

In [29]:
# RMSE_test_all['RF']    RMSE_test_all['SVR']    RMSE_test_all['XGB']    RMSE_test_all['LR']
# MAPE_test_all['RF']    MAPE_test_all['SVR']    MAPE_test_all['XGB']    MAPE_test_all['LR']
# diff_test_all['RF']    diff_test_all['SVR']    diff_test_all['XGB']    diff_test_all['LR']



# RMSE 간 차이가 있는지를 확인
print("-----RMSE-----")
print(Wiltest(RMSE_test_all['RF'],RMSE_test_all['SVR']))
print(Wiltest(RMSE_test_all['RF'],RMSE_test_all['XGB']))
print(Wiltest(RMSE_test_all['RF'],RMSE_test_all['LR']))
print(Wiltest(RMSE_test_all['SVR'],RMSE_test_all['XGB']))
print(Wiltest(RMSE_test_all['SVR'],RMSE_test_all['LR']))
print(Wiltest(RMSE_test_all['XGB'],RMSE_test_all['LR']))

# MAPE 간 차이가 있는지를 확인
print("-----MAPE-----")
print(Wiltest(MAPE_test_all['RF'],MAPE_test_all['SVR']))
print(Wiltest(MAPE_test_all['RF'],MAPE_test_all['XGB']))
print(Wiltest(MAPE_test_all['RF'],MAPE_test_all['LR']))
print(Wiltest(MAPE_test_all['SVR'],MAPE_test_all['XGB']))
print(Wiltest(MAPE_test_all['SVR'],MAPE_test_all['LR']))
print(Wiltest(MAPE_test_all['XGB'],MAPE_test_all['LR']))

# diff 간 차이가 있는지를 확인
print("-----diff-----")
print(Wiltest(diff_test_all['RF'],diff_test_all['SVR']))
print(Wiltest(diff_test_all['RF'],diff_test_all['XGB']))
print(Wiltest(diff_test_all['RF'],diff_test_all['LR']))
print(Wiltest(diff_test_all['SVR'],diff_test_all['XGB']))
print(Wiltest(diff_test_all['SVR'],diff_test_all['LR']))
print(Wiltest(diff_test_all['XGB'],diff_test_all['LR']))

-----RMSE-----
True
True
True
True
True
True
-----MAPE-----
True
True
True
True
True
True
-----diff-----
True
True
False
True
True
True


결과: RMSE 든 MAPE 든 모델 간 성능 차이가 확실하다.